<h2>Import different libraries </h2> 

In [ ]:
import os 
import time
import requests 
from requests_html import HTML 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

Define working directory

In [38]:
cwd = os.getcwd() # dirname of abs(path(__file__))
BASE_DIR = os.path.dirname(cwd)
BASE_DIR

'/home/vianney/projects/data-science-projects/notebooks'

In [39]:
options= Options()
options.add_argument(" --headless")
driver = webdriver.Chrome(options=options)

Set up product categories

In [40]:
categories = [
    "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/",
    "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/",
    "https://www.amazon.com/Best-Sellers/zgbs/fashion/"
    
]
    

In [41]:
categories

['https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/',
 'https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/',
 'https://www.amazon.com/Best-Sellers/zgbs/fashion/']

In [42]:
title_lookup = "#productTitle"
price_lookup = "#//span[@class='a-price a-text-price a-size-medium apexPriceToPay']"

In [43]:
first_url = categories[0] 

In [44]:
driver.get(first_url)

In [46]:
body_el = driver.find_element(By.CSS_SELECTOR,"body")
html_str = body_el.get_attribute("innerHTML")
html_obj = HTML(html=html_str)

In [48]:
#html_obj.links

In [49]:
new_links = [x for x in html_obj.links if x.startswith("/")]
#new_links = [x for x in new_links if "product_reviews/" not in x]
#print(new_links)

In [50]:
page_links = [f"https://www.amazon.com{x}" for x in new_links]
#page_links

Define a function to scrapea product given url, title and the price

In [51]:
def scrape_product_page(url, title_lookup = "#productTitle", price_lookup = "#priceblock_ourprice"):
    driver.get(url)
    time.sleep(1.2)
    body_el = driver.find_element(By.CSS_SELECTOR,"body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = driver.find_element(By.ID,'productTitle').text
    product_price = driver.find_element(By.XPATH,"//span[@class='a-price a-text-price a-size-medium apexPriceToPay']").text
    return product_title , product_price

In [52]:
first_product_page = page_links[0]
first_product_page

'https://www.amazon.com/product-reviews/B07TJ6PDB9/ref=zg_bs_toys-and-games_cr_26/145-7140083-6798014?ie=UTF8&refRID=CM8EHKJ6G1ZB7P50PKF9'

In [53]:
first_product_page = page_links[0]
first_product_page

'https://www.amazon.com/product-reviews/B07TJ6PDB9/ref=zg_bs_toys-and-games_cr_26/145-7140083-6798014?ie=UTF8&refRID=CM8EHKJ6G1ZB7P50PKF9'

Dealing with product title and price

In [54]:
for link in page_links:
    title, price = (None, None)
    try:
         title, price = scrape_product_page(link)
    except:
        pass
    if title != None and price != None:
        print(link, title, price)

https://www.amazon.com/TeeTurtle-Reversible-Octopus-Plushie-Patented/dp/B092P3LW22/ref=zg_bs_toys-and-games_2/145-7140083-6798014?_encoding=UTF8&psc=1&refRID=CM8EHKJ6G1ZB7P50PKF9 TeeTurtle | The Moody Reversible Octopus Plushie | Patented Design | Sensory Fidget Toy for Stress Relief | Light Pink + Pink | Angry + Rage | Show Your Mood Without Saying a Word! $15.00
https://www.amazon.com/Polly-Pocket-Surprises-Discover-Princess/dp/B084L1HW5P/ref=zg_bs_toys-and-games_5/145-7140083-6798014?_encoding=UTF8&psc=1&refRID=CM8EHKJ6G1ZB7P50PKF9 Polly Pocket Unicorn Party Large Compact Playset with Micro Polly & Lila Dolls, 25+ Surprises to Discover & Fun Princess Party Play Areas: Bouncy House, Castle, Swings, Water Floatie & More $24.99
https://www.amazon.com/Taco-Cat-Goat-Cheese-Pizza/dp/B077Z1R28P/ref=zg_bs_toys-and-games_11/145-7140083-6798014?_encoding=UTF8&psc=1&refRID=CM8EHKJ6G1ZB7P50PKF9 Taco Cat Goat Cheese Pizza $9.99
https://www.amazon.com/Seckton-Upgrade-Birthday-Portable-Card-Blue/d

<p> We'll use regular expression to work url</p>

In [ ]:


#<base-url>/<slug>/dp/>product_id>/

<h3> Using RegEx to parse url <h3>

<p> Here we set up url with regular expression to deal with product category.</p>

In [ ]:
r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/((?P<product_id>[\w-]+)/"